In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [55]:
pos_cash = pd.read_csv('D:/Data group project DPV/POS_CASH_balance.csv')
app_train = pd.read_csv('D:/Data group project DPV/application_train.csv')

**POS_CASH_balance.csv**

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample.

**POS_CASH_balance.csv FEATURE DESCRIPTION**

SK_ID_PREV: The ID of previous credit in Home Credit related to loan in our sample

SK_ID_CURR: ID of loan in our sample

MONTHS_BALANCE: Month of balance relative to application date

CNT_INSTALMENT: Term of previous credit (can change over time)

CNT_INSTALMENT_FUTURE: Installments left to pay on the previous credit

NAME_CONTRACT_STATUS: Contract status during the month

SK_DPD: DPD (days past due) during the month of previous credit

SK_DPD_DEF: DPD during the month with tolerance (debts with low loan amounts are ignored) of the previous credit

In [56]:
pos_cash.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
5,2207092,342166,-32,12.0,12.0,Active,0,0
6,1110516,204376,-38,48.0,43.0,Active,0,0
7,1387235,153211,-35,36.0,36.0,Active,0,0
8,1220500,112740,-31,12.0,12.0,Active,0,0
9,2371489,274851,-32,24.0,16.0,Active,0,0


## **FEATURE ADDED**

In [57]:
pos_cash_new = pos_cash.groupby(['SK_ID_PREV']).last().reset_index()
pos_cash_new.rename(index=str,
                    columns={'CNT_INSTALMENT_FUTURE': 'pos_cash_remaining_installments'},
                    inplace=True)

In [58]:
pos_cash['is_contract_status_completed'] = pos_cash['NAME_CONTRACT_STATUS'] == 'Completed'
group_object = pos_cash.groupby(['SK_ID_PREV'])['is_contract_status_completed'].sum().reset_index()
group_object.rename(index=str,
                    columns={'is_contract_status_completed': 'pos_cash_completed_contracts'},
                    inplace=True)
pos_cash_new = pos_cash_new.merge(group_object, on=['SK_ID_PREV'], how='left')

In [59]:
pos_cash_new

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,pos_cash_remaining_installments,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,pos_cash_completed_contracts
0,1000001,158271,-9,12.0,11.0,Active,0,0,1
1,1000002,101962,-54,4.0,4.0,Active,0,0,1
2,1000003,252457,-2,12.0,10.0,Active,0,0,0
3,1000004,260094,-29,10.0,10.0,Active,0,0,1
4,1000005,176456,-50,10.0,4.0,Active,0,0,1
...,...,...,...,...,...,...,...,...,...
936320,2843494,292375,-26,48.0,48.0,Active,0,0,1
936321,2843495,260963,-15,60.0,59.0,Active,0,0,1
936322,2843497,451578,-4,24.0,7.0,Active,0,0,0
936323,2843498,393881,-44,36.0,32.0,Active,0,0,1


In [68]:
agg_pos_cash = pos_cash_new.groupby(['SK_ID_PREV'])['MONTHS_BALANCE', 'CNT_INSTALMENT',\
                                                  'pos_cash_remaining_installments',\
                                                   'pos_cash_completed_contracts',\
                                                   'SK_DPD', 'SK_DPD_DEF'].agg( ['min', 'max', 'mean'])

<ipython-input-68-b7b410dc7a21>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_pos_cash = pos_cash_new.groupby(['SK_ID_PREV'])['MONTHS_BALANCE', 'CNT_INSTALMENT',\


In [69]:
agg_pos_cash

MONTHS_BALANCE          CNT_INSTALMENT              \
                      min max mean            min   max  mean   
SK_ID_PREV                                                      
1000001                -9  -9   -9           12.0  12.0  12.0   
1000002               -54 -54  -54            4.0   4.0   4.0   
1000003                -2  -2   -2           12.0  12.0  12.0   
1000004               -29 -29  -29           10.0  10.0  10.0   
1000005               -50 -50  -50           10.0  10.0  10.0   
...                   ...  ..  ...            ...   ...   ...   
2843494               -26 -26  -26           48.0  48.0  48.0   
2843495               -15 -15  -15           60.0  60.0  60.0   
2843497                -4  -4   -4           24.0  24.0  24.0   
2843498               -44 -44  -44           36.0  36.0  36.0   
2843499               -36 -36  -36           60.0  60.0  60.0   

           pos_cash_remaining_installments              \
                                       min   max  mean   
SK_ID_PREV                                               
1000001                               11.0  11.0  11.0   
1000002                                4.0   4.0   4.0   
1000003                               10.0  10.0  10.0   
1000004                               10.0  10.0  10.0   
1000005                                4.0   4.0   4.0   
...                                    ...   ...   ...   
2843494                               48.0  48.0  48.0   
2843495                               59.0  59.0  59.0   
2843497                                7.0   7.0   7.0   
2843498                               32.0  32.0  32.0   
2843499                               56.0  56.0  56.0   

           pos_cash_completed_contracts          SK_DPD          SK_DPD_DEF  \
                                    min max mean    min max mean        min   
SK_ID_PREV                                                                    
1000001                               1   1    1      0   0    0          0   
1000002                               1   1    1      0   0    0          0   
1000003                               0   0    0      0   0    0          0   
1000004                               1   1    1      0   0    0          0   
1000005                               1   1    1      0   0    0          0   
...                                 ...  ..  ...    ...  ..  ...        ...   
2843494                               1   1    1      0   0    0          0   
2843495                               1   1    1      0   0    0          0   
2843497                               0   0    0      0   0    0          0   
2843498                               1   1    1      0   0    0          0   
2843499                               1   1    1      0   0    0          0   

                     
           max mean  
SK_ID_PREV           
1000001      0    0  
1000002      0    0  
1000003      0    0  
1000004      0    0  
1000005      0    0  
...         ..  ...  
2843494      0    0  
2843495      0    0  
2843497      0    0  
2843498      0    0  
2843499      0    0  

[936325 rows x 18 columns]

In [70]:
agg_pos_cash.columns = ['POS_CAS_' + c[0] + '_' + c[1].upper() for c in agg_pos_cash.columns.values.tolist()]

In [71]:
agg_pos_cash

,POS_CAS_MONTHS_BALANCE_MIN,POS_CAS_MONTHS_BALANCE_MAX,POS_CAS_MONTHS_BALANCE_MEAN,POS_CAS_CNT_INSTALMENT_MIN,POS_CAS_CNT_INSTALMENT_MAX,POS_CAS_CNT_INSTALMENT_MEAN,POS_CAS_pos_cash_remaining_installments_MIN,POS_CAS_pos_cash_remaining_installments_MAX,POS_CAS_pos_cash_remaining_installments_MEAN,POS_CAS_pos_cash_completed_contracts_MIN,POS_CAS_pos_cash_completed_contracts_MAX,POS_CAS_pos_cash_completed_contracts_MEAN,POS_CAS_SK_DPD_MIN,POS_CAS_SK_DPD_MAX,POS_CAS_SK_DPD_MEAN,POS_CAS_SK_DPD_DEF_MIN,POS_CAS_SK_DPD_DEF_MAX,POS_CAS_SK_DPD_DEF_MEAN
SK_ID_PREV,,,,,,,,,,,,,,,,,,
1000001,-9,-9,-9,12.0,12.0,12.0,11.0,11.0,11.0,1,1,1,0,0,0,0,0,0
1000002,-54,-54,-54,4.0,4.0,4.0,4.0,4.0,4.0,1,1,1,0,0,0,0,0,0
1000003,-2,-2,-2,12.0,12.0,12.0,10.0,10.0,10.0,0,0,0,0,0,0,0,0,0
1000004,-29,-29,-29,10.0,10.0,10.0,10.0,10.0,10.0,1,1,1,0,0,0,0,0,0
1000005,-50,-50,-50,10.0,10.0,10.0,4.0,4.0,4.0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843494,-26,-26,-26,48.0,48.0,48.0,48.0,48.0,48.0,1,1,1,0,0,0,0,0,0
2843495,-15,-15,-15,60.0,60.0,60.0,59.0,59.0,59.0,1,1,1,0,0,0,0,0,0
2843497,-4,-4,-4,24.0,24.0,24.0,7.0,7.0,7.0,0,0,0,0,0,0,0,0,0


In [72]:
agg_pos_cash.isna().sum()

POS_CAS_MONTHS_BALANCE_MIN                        0
POS_CAS_MONTHS_BALANCE_MAX                        0
POS_CAS_MONTHS_BALANCE_MEAN                       0
POS_CAS_CNT_INSTALMENT_MIN                      890
POS_CAS_CNT_INSTALMENT_MAX                      890
POS_CAS_CNT_INSTALMENT_MEAN                     890
POS_CAS_pos_cash_remaining_installments_MIN     890
POS_CAS_pos_cash_remaining_installments_MAX     890
POS_CAS_pos_cash_remaining_installments_MEAN    890
POS_CAS_pos_cash_completed_contracts_MIN          0
POS_CAS_pos_cash_completed_contracts_MAX          0
POS_CAS_pos_cash_completed_contracts_MEAN         0
POS_CAS_SK_DPD_MIN                                0
POS_CAS_SK_DPD_MAX                                0
POS_CAS_SK_DPD_MEAN                               0
POS_CAS_SK_DPD_DEF_MIN                            0
POS_CAS_SK_DPD_DEF_MAX                            0
POS_CAS_SK_DPD_DEF_MEAN                           0
dtype: int64

In [73]:
agg_pos_cash.fillna(0, inplace = True)
agg_pos_cash.reset_index(inplace = True)

In [74]:
agg_pos_cash
# POS_CASH_balance.csv

,SK_ID_PREV,POS_CAS_MONTHS_BALANCE_MIN,POS_CAS_MONTHS_BALANCE_MAX,POS_CAS_MONTHS_BALANCE_MEAN,POS_CAS_CNT_INSTALMENT_MIN,POS_CAS_CNT_INSTALMENT_MAX,POS_CAS_CNT_INSTALMENT_MEAN,POS_CAS_pos_cash_remaining_installments_MIN,POS_CAS_pos_cash_remaining_installments_MAX,POS_CAS_pos_cash_remaining_installments_MEAN,POS_CAS_pos_cash_completed_contracts_MIN,POS_CAS_pos_cash_completed_contracts_MAX,POS_CAS_pos_cash_completed_contracts_MEAN,POS_CAS_SK_DPD_MIN,POS_CAS_SK_DPD_MAX,POS_CAS_SK_DPD_MEAN,POS_CAS_SK_DPD_DEF_MIN,POS_CAS_SK_DPD_DEF_MAX,POS_CAS_SK_DPD_DEF_MEAN
0,1000001,-9,-9,-9,12.0,12.0,12.0,11.0,11.0,11.0,1,1,1,0,0,0,0,0,0
1,1000002,-54,-54,-54,4.0,4.0,4.0,4.0,4.0,4.0,1,1,1,0,0,0,0,0,0
2,1000003,-2,-2,-2,12.0,12.0,12.0,10.0,10.0,10.0,0,0,0,0,0,0,0,0,0
3,1000004,-29,-29,-29,10.0,10.0,10.0,10.0,10.0,10.0,1,1,1,0,0,0,0,0,0
4,1000005,-50,-50,-50,10.0,10.0,10.0,4.0,4.0,4.0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936320,2843494,-26,-26,-26,48.0,48.0,48.0,48.0,48.0,48.0,1,1,1,0,0,0,0,0,0
936321,2843495,-15,-15,-15,60.0,60.0,60.0,59.0,59.0,59.0,1,1,1,0,0,0,0,0,0
936322,2843497,-4,-4,-4,24.0,24.0,24.0,7.0,7.0,7.0,0,0,0,0,0,0,0,0,0
936323,2843498,-44,-44,-44,36.0,36.0,36.0,32.0,32.0,32.0,1,1,1,0,0,0,0,0,0


### **Những feature được tạo thêm**

**pos_cash_remaining_installments:** số khoản vay còn phải trả

**pos_cash_completed_contracts:** số khoản vay đã được hoàn thành

### **Dấu hiệu nhận biết bảng POS_CASH_balance.csv: POS_CAS_**

In [75]:
agg_pos_cash.to_csv('C:/Users/Admin/Data prepare and Visualization/Group project/FINAL_POS_CASH_balance_df.csv')